In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  ca-certificates      anaconda::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi               anaconda::certifi-2020.4.5.1-py36_0 --> conda-forge::certifi-2020.4.5.1-py36h9f0ad1d_0
  openssl               anaconda::openssl-1.1.1g-h7b6447c_0 --> conda-forge::openssl-1.1.1g-h516909a_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Libraries imported.


<h2>INSTALLING lxml to parse HTML </h2>

In [3]:
conda install -c anaconda lxml 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    libxslt-1.1.33             |       h7d1a2b0_0         577 KB  anaconda
    lxml-4.5.0                 |   py36hefd8a0e_0         1.6 MB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    ------------------------------------------------------------
                                           Total:         6.2 MB

The following NEW packages will be INSTALLED:

  libxslt            anaconda/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               anaconda/linux-64::lxml-4.5.0-py36hefd8a0e_0

The following packages wi

In [2]:
#read the tables on the wikipedia page
dfs=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
data=dfs[0]
data.head(5)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
#copying data into dataframe
data=pd.DataFrame(dfs[0])

In [4]:
#data.drop(labels='Not Assigned',axis=0,inplace=True)
#data=data[data['Borough']!='Not Assigned']
type(data)

pandas.core.frame.DataFrame

In [5]:
data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
#deleting entries that don't have a Borough assigned
data=data[data.Borough!='Not assigned']
data.head()
#type(data)
#data.reset_index(drop=True)
#data.head()
data.reset_index(inplace=True)
data.head(2)
data.drop('index',inplace=True,axis=1)

In [7]:
data.drop('level_0',inplace=True,axis=1)
data.head(2)

KeyError: "['level_0'] not found in axis"

In [9]:
#data.drop('index',axis=1,inplace=True)
#data.drop('level_0',axis=1,inplace=True)
data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
#reading coordinates from CSV file
df=pd.read_csv('Geospatial_Coordinates.csv')
df.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df.shape

(103, 3)

In [12]:
#data.drop('Latitude',axis=1,inplace=True)
data.head(7)

df.loc[0,'Postal Code']
df.head(2)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


In [13]:
#Matching coordinates with Boroughs and adding Langitudes/Longitudes

pcode=data['Postal Code'].tolist()
a=[]
data['Latitude']=" "
data['Longitude']=" "
k=0
for val,k in zip(pcode,range(0,103)):
    for i in range(0,103):
        if val==df.loc[i,'Postal Code']:
            data.loc[k,'Latitude']=df.loc[i,'Latitude']
            data.loc[k,'Longitude']=df.loc[i,'Longitude']
               

In [14]:
#Seeing new updated dataframe
data.head(3)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606


In [15]:
#Getting the location for Toronto, Canada

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude

longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map which are Boroughs
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
CLIENT_ID = 'RP4H3ELE03XVE25W5PH2BSAXUT402P1LQLYAY0WSGBZWPZ3N' # your Foursquare ID
CLIENT_SECRET = 'ZAJD4T4W0HDPG3UIQJASKH4NEFBPQZYBBD4QXURLASJ1VPFU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: RP4H3ELE03XVE25W5PH2BSAXUT402P1LQLYAY0WSGBZWPZ3N
CLIENT_SECRET:ZAJD4T4W0HDPG3UIQJASKH4NEFBPQZYBBD4QXURLASJ1VPFU


In [18]:
neighborhood_latitude = data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = data.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of North York are 43.7532586, -79.3296565.


In [19]:
#Generating url to make calls

LIMIT=100
radius=1000
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
neighborhood_latitude,
neighborhood_longitude,
radius,
LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RP4H3ELE03XVE25W5PH2BSAXUT402P1LQLYAY0WSGBZWPZ3N&client_secret=ZAJD4T4W0HDPG3UIQJASKH4NEFBPQZYBBD4QXURLASJ1VPFU&v=20180605&ll=43.7532586,-79.3296565&radius=1000&limit=100'

In [20]:
#making request calls
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec7062aedbcad00215efcea'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 43.762258609000014,
    'lng': -79.31721997969855},
   'sw': {'lat': 43.74425859099999, 'lng': -79.34209302030145}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8991cbf964a520814232e3',
       'name': "Allwyn's Bakery",
       'location': {'address': '81 Underhill drive',
        'lat': 43.75984035203157,
        'lng': -79.32471879917513,
        'labeledLatLngs': [{'label': 'display'

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Brookbanks Park,Park,43.751976,-79.332140
2,Tim Hortons,Café,43.760668,-79.326368
3,A&W,Fast Food Restaurant,43.760643,-79.326865
4,Bruno's valu-mart,Grocery Store,43.746143,-79.324630


In [23]:
nearby_venues.shape

(29, 4)

In [24]:
#generating function to get nearby venues of boroughs
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
#creating dataframe to store nearby venues
toronto_venues = getNearbyVenues(names=data['Borough'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )


North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toron

In [26]:
toronto_venues.shape

(4912, 7)

In [27]:
toronto_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,North York,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,North York,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,North York,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


In [28]:
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,561,561,561,561,561,561
Downtown Toronto,1671,1671,1671,1671,1671,1671
East Toronto,412,412,412,412,412,412
East York,253,253,253,253,253,253
Etobicoke,261,261,261,261,261,261
Mississauga,53,53,53,53,53,53
North York,646,646,646,646,646,646
Scarborough,385,385,385,385,385,385
West Toronto,544,544,544,544,544,544


In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 333 uniques categories.


In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie

In [31]:
toronto_onehot.shape

(4912, 334)

In [32]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie

In [33]:
#printing top 5 venues from boroughs
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    

----Central Toronto----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.06
2                Café  0.05
3    Sushi Restaurant  0.04
4                 Gym  0.03


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.06
2  Japanese Restaurant  0.03
3                 Park  0.03
4           Restaurant  0.03


----East Toronto----
         venue  freq
0  Coffee Shop  0.06
1         Café  0.04
2      Brewery  0.03
3       Bakery  0.03
4         Park  0.03


----East York----
              venue  freq
0       Coffee Shop  0.09
1       Pizza Place  0.04
2              Café  0.03
3     Grocery Store  0.03
4  Greek Restaurant  0.03


----Etobicoke----
           venue  freq
0           Park  0.07
1    Pizza Place  0.06
2    Coffee Shop  0.05
3  Grocery Store  0.04
4       Pharmacy  0.04


----Mississauga----
                       venue  freq
0                Coffee Shop  0.11
1  Middle Eastern Restaurant  0.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
#getting top 10 venues from all boroughs
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Restaurant,Pizza Place,Gym,Park,Grocery Store,Sandwich Place
1,Downtown Toronto,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery
2,East Toronto,Coffee Shop,Café,Greek Restaurant,Pizza Place,Bakery,Pub,Park,Brewery,Italian Restaurant,Bar
3,East York,Coffee Shop,Pizza Place,Greek Restaurant,Café,Grocery Store,Bank,Sandwich Place,Indian Restaurant,Brewery,Park
4,Etobicoke,Park,Pizza Place,Coffee Shop,Grocery Store,Pharmacy,Restaurant,Bank,Liquor Store,Italian Restaurant,Shopping Mall


In [36]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 3, 3, 2, 0, 2, 2, 1, 4], dtype=int32)

In [37]:
# add clustering labels
borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged =data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(borough_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7533,-79.3297,2,Coffee Shop,Pizza Place,Park,Bank,Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy
1,M4A,North York,Victoria Village,43.7259,-79.3156,2,Coffee Shop,Pizza Place,Park,Bank,Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648,2,Coffee Shop,Pizza Place,Park,Bank,Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery


In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>EXAMINING THE CLUSTERS</h2>

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,Mississauga,0,Coffee Shop,Middle Eastern Restaurant,Hotel,Gym,Asian Restaurant,Bakery,Burrito Place,Bus Station,Mexican Restaurant,Indian Restaurant


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery
4,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery
9,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery
15,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery
20,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery
24,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery
25,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery
30,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery
31,West Toronto,1,Café,Coffee Shop,Bar,Bakery,Restaurant,Park,Italian Restaurant,Pizza Place,Sushi Restaurant,Thai Restaurant
36,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Gastropub,Theater,Hotel,Italian Restaurant,Bakery


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Coffee Shop,Pizza Place,Park,Bank,Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy
1,North York,2,Coffee Shop,Pizza Place,Park,Bank,Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy
3,North York,2,Coffee Shop,Pizza Place,Park,Bank,Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy
5,Etobicoke,2,Park,Pizza Place,Coffee Shop,Grocery Store,Pharmacy,Restaurant,Bank,Liquor Store,Italian Restaurant,Shopping Mall
6,Scarborough,2,Chinese Restaurant,Coffee Shop,Pizza Place,Fast Food Restaurant,Bakery,Bank,Park,Grocery Store,Restaurant,Sandwich Place
7,North York,2,Coffee Shop,Pizza Place,Park,Bank,Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy
10,North York,2,Coffee Shop,Pizza Place,Park,Bank,Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy
11,Etobicoke,2,Park,Pizza Place,Coffee Shop,Grocery Store,Pharmacy,Restaurant,Bank,Liquor Store,Italian Restaurant,Shopping Mall
12,Scarborough,2,Chinese Restaurant,Coffee Shop,Pizza Place,Fast Food Restaurant,Bakery,Bank,Park,Grocery Store,Restaurant,Sandwich Place
13,North York,2,Coffee Shop,Pizza Place,Park,Bank,Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,3,Coffee Shop,Pizza Place,Greek Restaurant,Café,Grocery Store,Bank,Sandwich Place,Indian Restaurant,Brewery,Park
14,East York,3,Coffee Shop,Pizza Place,Greek Restaurant,Café,Grocery Store,Bank,Sandwich Place,Indian Restaurant,Brewery,Park
19,East Toronto,3,Coffee Shop,Café,Greek Restaurant,Pizza Place,Bakery,Pub,Park,Brewery,Italian Restaurant,Bar
23,East York,3,Coffee Shop,Pizza Place,Greek Restaurant,Café,Grocery Store,Bank,Sandwich Place,Indian Restaurant,Brewery,Park
29,East York,3,Coffee Shop,Pizza Place,Greek Restaurant,Café,Grocery Store,Bank,Sandwich Place,Indian Restaurant,Brewery,Park
35,East York,3,Coffee Shop,Pizza Place,Greek Restaurant,Café,Grocery Store,Bank,Sandwich Place,Indian Restaurant,Brewery,Park
41,East Toronto,3,Coffee Shop,Café,Greek Restaurant,Pizza Place,Bakery,Pub,Park,Brewery,Italian Restaurant,Bar
47,East Toronto,3,Coffee Shop,Café,Greek Restaurant,Pizza Place,Bakery,Pub,Park,Brewery,Italian Restaurant,Bar
54,East Toronto,3,Coffee Shop,Café,Greek Restaurant,Pizza Place,Bakery,Pub,Park,Brewery,Italian Restaurant,Bar
100,East Toronto,3,Coffee Shop,Café,Greek Restaurant,Pizza Place,Bakery,Pub,Park,Brewery,Italian Restaurant,Bar


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,York,4,Coffee Shop,Pizza Place,Grocery Store,Furniture / Home Store,Park,Sandwich Place,Pharmacy,Café,Convenience Store,Gas Station
21,York,4,Coffee Shop,Pizza Place,Grocery Store,Furniture / Home Store,Park,Sandwich Place,Pharmacy,Café,Convenience Store,Gas Station
56,York,4,Coffee Shop,Pizza Place,Grocery Store,Furniture / Home Store,Park,Sandwich Place,Pharmacy,Café,Convenience Store,Gas Station
63,York,4,Coffee Shop,Pizza Place,Grocery Store,Furniture / Home Store,Park,Sandwich Place,Pharmacy,Café,Convenience Store,Gas Station
64,York,4,Coffee Shop,Pizza Place,Grocery Store,Furniture / Home Store,Park,Sandwich Place,Pharmacy,Café,Convenience Store,Gas Station
